<div style="clear: both;">
<div>
      <h2><b>EXPORT</b>4PHOBS</h2>
     <img src="https://github.com/MitjaGo/BAR/blob/main/logo-Adria.jpg?raw=1">
  </div>
</div>



In [ ]:
from IPython.display import IFrame
IFrame(src='https://docs.google.com/spreadsheets/d/15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk/edit?gid=1385640257#gid=1385640257', width=1200, height=600)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime

# ---------- Output container ----------
main_out = widgets.Output()

# ---------- Reload button ----------
reload_button = widgets.Button(
    description="Reload Content 🔄",
    button_style='warning',
    tooltip="Click to refresh all content",
    layout=widgets.Layout(width='200px')
)

# ---------- Function to generate dynamic content ----------
def create_widgets():
    # Format time as HH:MM:SS
    current_time = datetime.now().strftime("%H:%M:%S")
    w1 = widgets.Label(f"Refreshed at: {current_time}")
    return [w1]

# ---------- Function to populate content ----------
def initialize_content():
    with main_out:
        clear_output()
        widgets_list = create_widgets()
        for w in widgets_list:
            display(w)

# ---------- Button click callback ----------
def on_reload_click(b):
    initialize_content()

reload_button.on_click(on_reload_click)

# ---------- Display everything ----------
ui = widgets.VBox([reload_button, main_out])
display(ui)

# Initialize content
initialize_content()


In [ ]:
# @title
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
import base64
import urllib.parse

output = widgets.Output()

# --- Function to prepare PHOBS dataframe ---
def prepare_phobs_csv(df, hotel_id, los_code):
    if 'BAR' not in df.columns:
        df['BAR'] = 120  # default BAR value

    df['BAR'] = df['BAR'].apply(lambda x: f"BAR{x}")
    df['Hotel_ID'] = hotel_id
    df['nicla'] = 0
    df['Yield'] = f"YIELD{los_code}"

    if 'Datum' not in df.columns:
        df['Datum'] = pd.Timestamp.today().strftime('%Y-%m-%d')

    df = df[['Hotel_ID', 'Datum', 'nicla', 'BAR', 'Yield']]
    return df


# --- Function to create a download button (Voila-safe) ---
def create_download_button(df, filename, description):
    button = widgets.Button(
        description=description,
        button_style='info',
        layout=widgets.Layout(width="33%")
    )

    def on_click(b):
        with output:
            output.clear_output()
            csv_bytes = df.to_csv(index=False, header=False).encode()
            b64 = base64.b64encode(csv_bytes).decode()
            href = f"data:text/csv;base64,{b64}"
            html = f"""
            <div style='margin-top:10px;'>
                <b></b><br>
                <a download="{filename}.csv" href="{href}" target="_blank"
                   style='text-decoration:none; color:white; background-color:#5392ca; padding:6px 10px; border-radius:4px;'>
                   📥 Download {filename}.csv
                </a>
            </div>
            """
            display(HTML(html))

    button.on_click(on_click)
    return button


# --- Load master PHOBS sheet ---
gsheet_id = "15HJ7wxyUmo-gcl5_y1M9gl4Ti-JSsYEJZCjoI76s-Xk"
master_sheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet=PHOBS"
master_df = pd.read_csv(master_sheet_url)

# Generate hotels list
hotels = [(row['Hotel_Name'], row['Hotel_ID'], row['YIELD_Code']) for _, row in master_df.iterrows()]

buttons = []
with output:
    display(HTML("<b>Loading hotel sheets...</b>"))

# --- Load each hotel's sheet and create buttons ---
for hotel_name, hotel_id, los_code in hotels:
    try:
        sheet_name_encoded = urllib.parse.quote(hotel_name)
        sheet_url = f"https://docs.google.com/spreadsheets/d/{gsheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name_encoded}"
        df = pd.read_csv(sheet_url)
        df = prepare_phobs_csv(df, hotel_id, los_code)
        button = create_download_button(df, f"{hotel_name}-PHOBS", hotel_name.replace("_", " "))
        buttons.append(button)
    except Exception as e:
        with output:
            display(HTML(f"<b style='color:red'>Failed to load sheet for {hotel_name}:</b> {e}"))

# --- Layout buttons ---
rows = [widgets.HBox(buttons[i:i+3]) for i in range(0, len(buttons), 3)]
vbox = widgets.VBox(rows)

output.clear_output()
display(HTML("<h3>PHOBS BAR Export Generator</h3>"))
display(vbox, output)
